## Boosting
* 부스팅 알고리즘은 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 토앻 오류를 개선해 나가는 학습 방법
* weak learner → 과적합 피하기 쉬움
* 부스팅의 대표적인 구현은 AdaBoost(Adaptive boosting)와 그래디언트 부스트가 있음
* 그래디언트 부스팅 → 기울기 기반 오류 감소 모형   
일차 분류의 오류에 대해서 다시 분류, 반복
* XGB → 손실함수 지정, 오버피팅, 소요시간 감소

#### GBM Hyper Parameters
* loss : 경사 하강법에서 사용할 비용 함수 지정. 특별한 값 없으면 기본값인 'deviance'를 그대로 적용
* learning_rate : GBM이 학습을 진행할 때마다 적용하는 학습률. 0~1 사이의 값.    
너무 작으면 반복이 완료되어도 못 찾음, 반대로 너무 커도 오류 못 찾고 지나칠 수 있음.
* n_estimators : weak learner의 개수. 개수 많으면 일정 수준까지는 좋아질 수 있음.   
반면 개수가 많아지면 시간이 오래 걸림. 기본값 100
* subsample : 데이터 샘플링 비율. 기본값 1, 전체 기반 학습.     
0~1사이 설정 가능.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import time
from modules import HmDt
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = HmDt.get_human_dataset()

In [ ]:
# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print(f"GBM 정확도 : {gb_accuracy:.4f}")
print(f"GBM 수행시간 : {time.time() - start_time}")

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators' : [100, 500],
    'learning_rate' : [0.05, 0.1]
}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)
print(f"최적 하이퍼 파라미터 : {grid_cv.best_params_}")
print(f"최고 예측 정확도 : {grid_cv.best_score_:.4f}")

In [ ]:
# GridSearchCV를 이용하여 최적으로 학습된 estimator로 predict 수행
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print(f"GBM 정확도 : {gb_accuracy:.4f}")

## XGBoost

#### class xgboost.XGBRegressor(*, objective='reg:squarederror', **kwargs)
Parameters

-   **n_estimators**  ([_int_](https://docs.python.org/3.6/library/functions.html#int "(in Python v3.6)")) – Number of gradient boosted trees. Equivalent to number of boosting rounds.
    
-   **max_depth**  (_Optional__[_[_int_](https://docs.python.org/3.6/library/functions.html#int "(in Python v3.6)")_]_) – Maximum tree depth for base learners.
    
-   **learning_rate**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Boosting learning rate (xgb's "eta")
    
-   **verbosity**  (_Optional__[_[_int_](https://docs.python.org/3.6/library/functions.html#int "(in Python v3.6)")_]_) – The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
    
-   **objective**  ([_Union_](https://docs.python.org/3.6/library/typing.html#typing.Union "(in Python v3.6)")_[_[_str_](https://docs.python.org/3.6/library/stdtypes.html#str "(in Python v3.6)")_,_ [_Callable_](https://docs.python.org/3.6/library/typing.html#typing.Callable "(in Python v3.6)")_[__[_[_numpy.ndarray_](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray "(in NumPy v1.22)")_,_ [_numpy.ndarray_](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray "(in NumPy v1.22)")_]__,_ [_Tuple_](https://docs.python.org/3.6/library/typing.html#typing.Tuple "(in Python v3.6)")_[_[_numpy.ndarray_](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray "(in NumPy v1.22)")_,_ [_numpy.ndarray_](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray "(in NumPy v1.22)")_]__]__,_ _NoneType__]_) – Specify the learning task and the corresponding learning objective or a custom objective function to be used (see note below).
    
-   **booster**  (_Optional__[_[_str_](https://docs.python.org/3.6/library/stdtypes.html#str "(in Python v3.6)")_]_) – Specify which booster to use: gbtree, gblinear or dart.
    
-   **tree_method**  (_Optional__[_[_str_](https://docs.python.org/3.6/library/stdtypes.html#str "(in Python v3.6)")_]_) – Specify which tree method to use. Default to auto. If this parameter is set to default, XGBoost will choose the most conservative option available. It's recommended to study this option from the parameters document:  [https://xgboost.readthedocs.io/en/latest/treemethod.html](https://xgboost.readthedocs.io/en/latest/treemethod.html).
    
-   **n_jobs**  (_Optional__[_[_int_](https://docs.python.org/3.6/library/functions.html#int "(in Python v3.6)")_]_) – Number of parallel threads used to run xgboost. When used with other Scikit-Learn algorithms like grid search, you may choose which algorithm to parallelize and balance the threads. Creating thread contention will significantly slow down both algorithms.
    
-   **gamma**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Minimum loss reduction required to make a further partition on a leaf node of the tree.
    
-   **min_child_weight**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Minimum sum of instance weight(hessian) needed in a child.
    
-   **max_delta_step**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Maximum delta step we allow each tree's weight estimation to be.
    
-   **subsample**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Subsample ratio of the training instance.
    
-   **colsample_bytree**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Subsample ratio of columns when constructing each tree.
    
-   **colsample_bylevel**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Subsample ratio of columns for each level.
    
-   **colsample_bynode**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Subsample ratio of columns for each split.
    
-   **reg_alpha**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – L1 regularization term on weights (xgb's alpha).
    
-   **reg_lambda**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – L2 regularization term on weights (xgb's lambda).
    
-   **scale_pos_weight**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – Balancing of positive and negative weights.
    
-   **base_score**  (_Optional__[_[_float_](https://docs.python.org/3.6/library/functions.html#float "(in Python v3.6)")_]_) – The initial prediction score of all instances, global bias.
    
-   **random_state**  (_Optional__[__Union__[_[_numpy.random.RandomState_](https://numpy.org/doc/stable/reference/random/legacy.html#numpy.random.RandomState "(in NumPy v1.22)")_,_ [_int_](https://docs.python.org/3.6/library/functions.html#int "(in Python v3.6)")_]__]_) –

In [ ]:
import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, \
                            recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
from modules import Eval

import warnings
warnings.filterwarnings("ignore")

data_set = load_breast_cancer()
brst_data = data_set.data
brst_labels = data_set.target

brst_df = pd.DataFrame(data=brst_data, columns=data_set.feature_names)
brst_df["target"] = brst_labels

brst_df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(brst_data, brst_labels,
                                                    test_size=0.3, random_state=121)

brst_xgb_clt = xgb.XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
brst_xgb_clt.fit(X_train, y_train)

brst_pred = brst_xgb_clt.predict(X_test)
brst_pred_proba = brst_xgb_clt.predict_proba(X_test)

Eval.get_clf_eval(y_test, brst_pred, brst_pred_proba)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 12))

plot_importance(brst_xgb_clt, ax=ax)

### XGBoost Early Stop

In [ ]:
brst_xgb_clt_es = xgb.XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
brst_xgb_clt_es.fit(X_train, y_train, early_stopping_rounds=5, eval_metric="logloss", eval_set=[(X_test, y_test)])

brst_pred_es = brst_xgb_clt_es.predict(X_test)
brst_pred_proba_es = brst_xgb_clt_es.predict_proba(X_test)

Eval.get_clf_eval(y_test, brst_pred_es, brst_pred_proba_es)

## SVM
* SVM은 분류에 사용되는 지도학습 머신러닝 모델이다.
* SVM은 서포트 벡터(support vectors)를 사용해서 결정 경계(Decision Boundary)를 정의하고, 분류되지 않은 점을 해당 결정 경계와 비교해서 분류한다.
* 서포트 벡터(support vectors)는 결정 경계에 가장 가까운 각 클래스의 점들이다.
* 서포트 벡터와 결정 경계 사이의 거리를 마진(margin)이라고 한다.
* SVM은 허용 가능한 오류 범위 내에서 가능한 최대 마진을 만들려고 한다.
* 파라미터 C는 허용되는 오류 양을 조절한다. C 값이 클수록 오류를 덜 허용하며 이를 하드 마진(hard margin)이라 부른다. 반대로 C 값이 작을수록 오류를 더 많이 허용해서 소프트 마진(soft margin)을 만든다.
* SVM에서는 선형으로 분리할 수 없는 점들을 분류하기 위해 커널(kernel)을 사용한다.
* 커널(kernel)은 원래 가지고 있는 데이터를 더 높은 차원의 데이터로 변환한다. 2차원의 점으로 나타낼 수 있는 데이터를 다항식(polynomial) 커널은 3차원으로, RBF 커널은 점을 무한한 차원으로 변환한다.
* RBF 커널에는 파라미터 감마(gamma)가 있다. 감마가 너무 크면 학습 데이터에 너무 의존해서 오버피팅이 발생할 수 있다.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X_iris = iris.data
y_iris = iris.target

X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(
    X_iris, y_iris, test_size=0.5
)

In [ ]:
from sklearn.svm import SVC

# 감마값이 작으면 언더피팅 위험  /  C 값이 크면 하드마진에 의한 오버피팅 위험
sv_clf = SVC(gamma=0.001, C=100.)
sv_clf.fit(X_iris_train, y_iris_train)

sv_pred = sv_clf.predict(X_iris_test)
print(y_iris_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_iris_test, sv_pred))

## KNN (k-Nearest Neighbour)
* 레이블 값을 미리 주는 지도학습
* 장점 : 어떤 종류의 학습이나 준비 시간이 필요 없음
* 단점 : 특징 공간에 있는 모든 데이터에 대한 정보가 필요함      
가장 가까운 이웃을 찾기 위해 새로운 데이터에서 모든 기존 데이터까지의 거리를 확인해야 하기 때문     
데이터와 클래스가 많이 있으면, 많은 메모리 공간과 계산 시간이 필요함

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

iris = load_iris()
sepal = iris.data[:, 0:2]
kind = iris.target

plt.xlabel("Sepal Length")
plt.ylabel("Sepal Width")
plt.plot(sepal[kind==0][:,0], sepal[kind==0][:,1],
        "ro", label="Setosa")
plt.plot(sepal[kind==1][:,0], sepal[kind==1][:,1],
        "bo", label="Versicolor")
plt.plot(sepal[kind==2][:,0], sepal[kind==2][:,1],
        "yo", label="Verginica")

plt.legend()

In [ ]:
X_i_knn = iris.data
y_i_knn = iris.target

X_i_knn_train, X_i_knn_test, y_i_knn_train, y_i_knn_test = train_test_split(
    X_i_knn, y_i_knn, test_size=0.2, random_state=121
)

print(X_i_knn_train.shape)
print(X_i_knn_test.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=6)

knn.fit(X_i_knn_train, y_i_knn_train)

knn_pred = knn.predict(X_i_knn_test)
print(accuracy_score(y_i_knn_test, knn_pred))

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors=5)

knn2.fit(X_i_knn, y_i_knn)

# 0 = setosa, 1 = versicolor, 2 = virginica
classes = {
    0 : "Setosa",
    1 : "Versicolor",
    2 : "Virginica"
}

# 새로운 데이터 제시
X_new = [[3,4,5,2],
        [5,4,2,2]]
y_predict = knn.predict(X_new)

print(classes[y_predict[0]])
print(classes[y_predict[1]])

## Pipeline
* 파이프라인을 사용하면 데이터 사전 처리 및 분류의 모든 단계를 포함하는 단일 개체를 만들 수 있다.

* train과 test 데이터 손실을 피할 수 있다.
* 교차 검증 및 기타 모델 선택 유형을 쉽게 만든다.
* 재현성 증가

In [ ]:
from sklearn.datasets import make_regression, make_classification

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

X_rand, y_rand = make_classification(n_samples=100, n_features=10, n_informative=2)

X_rand_train, X_rand_test, y_rand_train, y_rand_test = train_test_split(X_rand, y_rand,
                                                        test_size=0.3, random_state=121)

X_rand_train.shape, X_rand_test.shape, y_rand_train.shape, y_rand_test.shape

In [ ]:
# it takes a list of tuples parameter
pipline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression())
])

# use the pipeline object as you would do with a regular classifier
pipline.fit(X_rand_train, y_rand_train)

y_rand_pred = pipline.predict(X_rand_test)

accuracy_score(y_rand_test, y_rand_pred)